#### TestUse.ipynb

**Objective:** Rebuild the `FuzzyInferenceSystem` architecture from fuzzyR using Python and PyTorch to implement a basic fuzzy inference engine.

This notebook proceeds step-by-step:
1. Environment and dependency validation  
2. Definition of the `FuzzyInferenceSystem` class  
3. Core `eval()` inference workflow  
4. Basic input testing  

Later, each component will be modularized into Python files under the `src/` directory.


In [29]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys, os

print("Python interpreter path:", sys.executable)
print("Torch version:", torch.__version__)
print("Numpy version:", np.__version__)


Python interpreter path: c:\Users\admin\Desktop\fuzzyinferencesystem\fuzzy-gradient-optimisation\.venv\Scripts\python.exe
Torch version: 2.9.0+cpu
Numpy version: 2.3.4


#### FuzzyInferenceSystem Class Structure

In fuzzyR, a FIS (Fuzzy Inference System) object consists of:
- Input and output variables (each with membership functions)
- A rule base (if–then rules)
- Inference and aggregation methods (e.g., min, max, prod)

We first build a Python version of this structural framework.


In [17]:
import torch

class FuzzyInferenceSystem:
    """
    Python reimplementation of FuzzyR::fuzzyinferencesystem.R
    Logical order: newfis() → addvar() → addmf() → addrule() → evalfis()
    """

    def __init__(self, name, fis_type="mamdani", mf_type="t1",
                 and_method="min", or_method="max", imp_method="min",
                 agg_method="max", defuzz_method="centroid"):
        """Equivalent to fuzzyR::newfis()"""
        self.name = name
        self.type = fis_type
        self.mf_type = mf_type
        self.and_method = and_method
        self.or_method = or_method
        self.imp_method = imp_method
        self.agg_method = agg_method
        self.defuzz_method = defuzz_method
        self.input = []
        self.output = []
        self.rule = []
        print(f"Created FIS: {self.name} (type={self.type})")

    # --------------------------------------------------------------
    # addvar()
    # --------------------------------------------------------------
    def add_variable(self, var_type, name, var_range, method=None, params=None, firing_method="tnorm.min.max"):
        variable = {
            "name": name,
            "range": list(var_range),
            "method": method,
            "params": params,
            "mf": [],
            "firing_method": firing_method
        }
        if var_type == "input":
            self.input.append(variable)
        elif var_type == "output":
            self.output.append(variable)
        else:
            raise ValueError("var_type must be 'input' or 'output'")
        print(f"Added {var_type}: {name} range={var_range}")

    # --------------------------------------------------------------
    # addmf()
    # --------------------------------------------------------------
    def add_mf(self, var_type, var_index, mf_name, mf_type, mf_params):
        mf_data = {"name": mf_name, "type": mf_type, "params": mf_params}
        if var_type == "input":
            self.input[var_index]["mf"].append(mf_data)
        elif var_type == "output":
            self.output[var_index]["mf"].append(mf_data)
        else:
            raise ValueError("var_type must be 'input' or 'output'")
        print(f"Added MF '{mf_name}' ({mf_type}) to {var_type}[{var_index}]")

    # --------------------------------------------------------------
    # addrule()
    # --------------------------------------------------------------
    def add_rule(self, rule_list):
        self.rule.append(rule_list)
        print(f"Added rule: {rule_list}")

    # --------------------------------------------------------------
    # showrule()
    # --------------------------------------------------------------
    def show_rules(self):
        if not self.rule:
            print("No rules defined.")
            return
        print("=== Rule Base ===")
        for i, r in enumerate(self.rule, 1):
            print(f"{i}. Rule: {r}")
        print("=================")

    # --------------------------------------------------------------
    # evalfis() — PyTorch reimplementation
    # --------------------------------------------------------------
    def eval(self, inputs, point_n=1001):
        """
        PyTorch-based fuzzyR::evalfis equivalent.
        Steps:
          1. Fuzzification
          2. Rule evaluation (min/max + weight)
          3. Aggregation (max)
          4. Defuzzification (centroid)
        """
        print("=== Starting FIS Evaluation ===")

        # Convert inputs to tensor
        x = torch.tensor(inputs, dtype=torch.float32).flatten()

        # ---- Step 1: Fuzzification ----
        input_mfs = []
        for i, var in enumerate(self.input):
            var_mf_vals = []
            for mf in var["mf"]:
                params = torch.tensor(mf["params"], dtype=torch.float32)
                if mf["type"] == "gaussmf":
                    sigma, c = params
                    mu = torch.exp(-((x[i] - c) ** 2) / (2 * sigma ** 2))
                else:
                    raise NotImplementedError(f"MF type '{mf['type']}' not implemented.")
                var_mf_vals.append(mu)
            input_mfs.append(var_mf_vals)

        # ---- Step 2: Rule evaluation ----
        rule_strengths = []
        for rule in self.rule:
            antecedents = []
            for j in range(len(self.input)):  # input part
                val = rule[j]
                if val == 0:
                    continue
                mf_idx = abs(val) - 1
                mu = input_mfs[j][mf_idx]
                if val < 0:
                    mu = 1 - mu
                antecedents.append(mu)

            if rule[-1] == 1:  # AND
                firing = torch.min(torch.stack(antecedents))
            else:              # OR
                firing = torch.max(torch.stack(antecedents))

            
            weight = float(rule[-2])
            firing = firing * weight

            rule_strengths.append(firing)

        # ---- Step 3: Aggregation (with min implication, like FuzzyR) ----
        y = torch.linspace(self.output[0]["range"][0],
                           self.output[0]["range"][1], point_n, dtype=torch.float32)
        agg_mu = torch.zeros_like(y)

        for rule, firing in zip(self.rule, rule_strengths):
            out_idx = abs(rule[len(self.input)]) - 1
            out_mf = self.output[0]["mf"][out_idx]
            sigma, c = torch.tensor(out_mf["params"], dtype=torch.float32)
            mu = torch.exp(-((y - c) ** 2) / (2 * sigma ** 2))

            # --- "min" implication instead of "prod" ---
            mu = torch.minimum(mu, torch.tensor(firing))

            # --- Aggregate by "max" ---
            agg_mu = torch.maximum(agg_mu, mu)

        # ---- Step 4: Defuzzification (centroid) ----
        numerator = torch.sum(y * agg_mu)
        denominator = torch.sum(agg_mu) + 1e-8
        crisp_output = numerator / denominator

        print("=== Evaluation Completed ===")
        return crisp_output.item()

    # --------------------------------------------------------------
    # show.FIS.R
    # --------------------------------------------------------------
    def summary(self):
        print("=== Fuzzy Inference System Summary ===")
        print(f"Name: {self.name}")
        print(f"Type: {self.type}")
        print(f"Inputs: {len(self.input)}")
        print(f"Outputs: {len(self.output)}")
        print(f"Rules: {len(self.rule)}")
        print("======================================")


In [18]:

fis = FuzzyInferenceSystem("tipper")

# 输入：service (0–10)
fis.add_variable("input", "service", (0, 10))
fis.add_mf("input", 0, "poor", "gaussmf", [1.5, 0])
fis.add_mf("input", 0, "good", "gaussmf", [1.5, 10])

# 输出：tip (0–30)
fis.add_variable("output", "tip", (0, 30))
fis.add_mf("output", 0, "low", "gaussmf", [4, 0])
fis.add_mf("output", 0, "high", "gaussmf", [4, 30])

# 规则
fis.add_rule([2, 2, 1, 1])  # if good → high
fis.add_rule([1, 1, 1, 1])  # if poor → low

fis.show_rules()
fis.summary()

# 推理
result = fis.eval([7.0], point_n=101)
print(f"Output: {result:.4f}")


Created FIS: tipper (type=mamdani)
Added input: service range=(0, 10)
Added MF 'poor' (gaussmf) to input[0]
Added MF 'good' (gaussmf) to input[0]
Added output: tip range=(0, 30)
Added MF 'low' (gaussmf) to output[0]
Added MF 'high' (gaussmf) to output[0]
Added rule: [2, 2, 1, 1]
Added rule: [1, 1, 1, 1]
=== Rule Base ===
1. Rule: [2, 2, 1, 1]
2. Rule: [1, 1, 1, 1]
=== Fuzzy Inference System Summary ===
Name: tipper
Type: mamdani
Inputs: 1
Outputs: 1
Rules: 2
=== Starting FIS Evaluation ===
=== Evaluation Completed ===
Output: 25.1179


C:\Users\admin\AppData\Local\Temp\ipykernel_25936\2166283950.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mu = torch.minimum(mu, torch.tensor(firing))
